In [ ]:
# This notebook evaluates the testing times discussed in the paper, 
#
#     "ZMPY3D: Accelerating Protein Structure Volume Analysis through Vectorized 3D Zernike Moments and Python-based GPU Integration."
#
# The computation times for ZMPY3D packages in both CPU and GPU environments have been assessed.
# It requires the simultaneous installation of ZMPY3D_TF, ZMPY3D_CP, and ZMPY3D versions.
# Testing order: ZMPY3D_TF, followed by ZMPY3D_CP, and finally ZMPY3D."
#
# This notebook primarily consists of the following steps: 
#     1. Set the MaxOrder number to 20
#     2. Load precalculated cache
#     3. Create a callable function (including generating Zernike moments and normalization)
#     4. Generate a 100x100x100 voxel in CPU/GPU memory
#     5. Use the callable function 10,000 times to measure time.
#
# Keep in mind that GPUs have a warming-up time. It is recommended to run the cell multiple times  for a fair performance.



In [ ]:
# # Install all ZMPY3D versions for NumPy, CuPy, and TensorFlow.
# ! pip install -i https://test.pypi.org/simple/ ZMPY3D
# ! pip install -i https://test.pypi.org/simple/ ZMPY3D-CP
# ! pip install -i https://test.pypi.org/simple/ ZMPY3D-TF

# print(f"It is recommended to restart the Python kernel for the IPython notebook.")

In [ ]:
import ZMPY3D_TF as z
import tensorflow as tf
import pickle

MaxOrder = 20
Param=z.get_global_parameter()

# Find the cache_data directory based on the site package location of ZMPY3D.
LogCacheFilePath=z.__file__.replace('__init__.py', 'cache_data') + '/LogG_CLMCache_MaxOrder{:02d}.pkl'.format(MaxOrder)

with open(LogCacheFilePath, 'rb') as file:
    CachePKL = pickle.load(file)

# convert to tensorflow objects
GCache_pqr_linear= tf.convert_to_tensor(CachePKL['GCache_pqr_linear'])
GCache_complex= tf.convert_to_tensor(CachePKL['GCache_complex'])
GCache_complex_index= tf.convert_to_tensor(CachePKL['GCache_complex_index'])
CLMCache3D= tf.convert_to_tensor(CachePKL['CLMCache3D'],dtype=tf.complex128)
CLMCache= tf.convert_to_tensor(CachePKL['CLMCache'], dtype=tf.float64)

print(f"Now using the MaxOrder of {MaxOrder}.")
print(f"Pre-calculated parameters for Tensorflow have been loaded successfully.")

In [ ]:
@tf.function
def OneTimeConversion_TF(Voxel3D,MaxOrder):

    Dimension_BBox_scaled=tf.shape(Voxel3D)
    
    MaxOrder=tf.convert_to_tensor(MaxOrder,dtype=tf.int64)
    
    X_sample = tf.range(Dimension_BBox_scaled[0] + 1,dtype=tf.float64)
    Y_sample = tf.range(Dimension_BBox_scaled[1] + 1,dtype=tf.float64)
    Z_sample = tf.range(Dimension_BBox_scaled[2] + 1,dtype=tf.float64)
    
    [VolumeMass,Center,_]=z.calculate_bbox_moment(Voxel3D,1,X_sample,Y_sample,Z_sample)
    
    [AverageVoxelDist2Center,_]=z.calculate_molecular_radius(Voxel3D,Center,VolumeMass,tf.convert_to_tensor(Param['default_radius_multiplier'], dtype=tf.float64))
    
    Sphere_X_sample, Sphere_Y_sample, Sphere_Z_sample=z.get_bbox_moment_xyz_sample(Center,AverageVoxelDist2Center,Dimension_BBox_scaled)
    
    _,_,SphereBBoxMoment=z.calculate_bbox_moment(Voxel3D
                                      ,MaxOrder
                                      ,Sphere_X_sample
                                      ,Sphere_Y_sample
                                      ,Sphere_Z_sample)
    
    ZMoment_scaled,_=z.calculate_bbox_moment_2_zm(MaxOrder
                                       , GCache_complex
                                       , GCache_pqr_linear
                                       , GCache_complex_index
                                       , CLMCache3D
                                       , SphereBBoxMoment)
    
    ZM_3DZD_invariant=z.get_3dzd_121_descriptor(ZMoment_scaled)
    
    ZM_3DZD_invariant_121=tf.reshape(tf.boolean_mask(ZM_3DZD_invariant, ~tf.math.is_nan(ZM_3DZD_invariant)), [-1])
    return ZM_3DZD_invariant_121


print(f"Merge all steps into a single callable Tensorfow function, OneTimeConversion_TF, decorated with @tf.function for optimized graph compilation.")

In [ ]:
%%time

Voxel3D = tf.random.uniform([100, 100, 100], dtype=tf.float64)
print(f"Initialize a 100x100x100 matrix with random values in GPU memory.")

Repeat=10000

for _ in range(Repeat):
    OneTimeConversion_TF(Voxel3D,MaxOrder)

print(f"Iteratively invoke the OneTimeConversion_TF function {Repeat} times using a for loop.")
print(f'Noted: The OneTimeConversion_TF uses eager functions to deliver immediate feedback on demonstrations to developers.')
print(f"Time elapsed is as follows:")

In [ ]:
import ZMPY3D_CP as z
import cupy as cp
import pickle

MaxOrder = 20
Param=z.get_global_parameter()

# Find the cache_data directory based on the site package location of ZMPY3D.
LogCacheFilePath=z.__file__.replace('__init__.py', 'cache_data') + '/LogG_CLMCache_MaxOrder{:02d}.pkl'.format(MaxOrder)

with open(LogCacheFilePath, 'rb') as file:
    CachePKL = pickle.load(file)

# convert cache into cupy objects
GCache_pqr_linear= cp.array(CachePKL['GCache_pqr_linear'], dtype=cp.int32)
GCache_complex= cp.array(CachePKL['GCache_complex'],dtype=cp.complex128)
GCache_complex_index= cp.array(CachePKL['GCache_complex_index'], dtype=cp.int32)
CLMCache3D= cp.array(CachePKL['CLMCache3D'],dtype=cp.complex128)
CLMCache= cp.array(CachePKL['CLMCache'], dtype=cp.float64)

print(f"Now using the MaxOrder of {MaxOrder}.")
print(f"Pre-calculated parameters for CuPy have been loaded successfully.")

In [ ]:
def OneTimeConversion_CP(Voxel3D,MaxOrder):
    Dimension_BBox_scaled=cp.shape(Voxel3D) # cp.shape does not create a matrix on the GPU.
    Dimension_BBox_scaled=cp.array(Dimension_BBox_scaled,dtype=cp.int32)
    
    MaxOrder=cp.array(MaxOrder,dtype=cp.int64)
    
    X_sample = cp.arange(Dimension_BBox_scaled[0] + 1, dtype=cp.float64)
    Y_sample = cp.arange(Dimension_BBox_scaled[1] + 1, dtype=cp.float64)
    Z_sample = cp.arange(Dimension_BBox_scaled[2] + 1, dtype=cp.float64)
    
    [VolumeMass,Center,_]=z.calculate_bbox_moment(Voxel3D,1,X_sample,Y_sample,Z_sample)
    
    [AverageVoxelDist2Center,_]=z.calculate_molecular_radius(Voxel3D,Center,VolumeMass,cp.array(Param['default_radius_multiplier'], dtype=cp.float64))
    
    Sphere_X_sample, Sphere_Y_sample, Sphere_Z_sample=z.get_bbox_moment_xyz_sample(Center,AverageVoxelDist2Center,Dimension_BBox_scaled)
    
    _,_,SphereBBoxMoment=z.calculate_bbox_moment(Voxel3D
                                      ,MaxOrder
                                      ,Sphere_X_sample
                                      ,Sphere_Y_sample
                                      ,Sphere_Z_sample)
    
    
    ZMoment_scaled,_=z.calculate_bbox_moment_2_zm(MaxOrder
                                       , GCache_complex
                                       , GCache_pqr_linear
                                       , GCache_complex_index
                                       , CLMCache3D
                                       , SphereBBoxMoment)
    
    ZM_3DZD_invariant=z.get_3dzd_121_descriptor(ZMoment_scaled)
    
    ZM_3DZD_invariant_121=ZM_3DZD_invariant[~cp.isnan(ZM_3DZD_invariant)]
    return ZM_3DZD_invariant_121



print(f"Merge all steps into a single callable CuPy function, OneTimeConversion_CP.")


In [ ]:
%%time

Voxel3D=matrix = cp.random.rand(100, 100, 100)
print(f"Initialize a 100x100x100 matrix with random values in GPU memory.")

Repeat=10000
for _ in range(Repeat):
    OneTimeConversion_CP(Voxel3D,MaxOrder)

print(f"Iteratively invoke the OneTimeConversion_CP function {Repeat} times using a for loop.")
print(f"Time elapsed is as follows:")

In [ ]:
import ZMPY3D as z
import numpy as np
import pickle
import os

MaxOrder = 20

Param=z.get_global_parameter()

# Find the cache_data directory based on the site package location of ZMPY3D.
LogCacheFilePath=z.__file__.replace('__init__.py', 'cache_data') + '/LogG_CLMCache_MaxOrder{:02d}.pkl'.format(MaxOrder)

with open(LogCacheFilePath, 'rb') as file:
    CachePKL = pickle.load(file)

GCache_pqr_linear=CachePKL['GCache_pqr_linear']
GCache_complex=CachePKL['GCache_complex']
GCache_complex_index=CachePKL['GCache_complex_index']
CLMCache3D=CachePKL['CLMCache3D']
CLMCache=CachePKL['CLMCache']

print(f"Now using the MaxOrder of {MaxOrder}.")
print(f"Pre-calculated parameters for NumPy have been loaded successfully.")

In [ ]:
def OneTimeConversion(Voxel3D,MaxOrder):
    Dimension_BBox_scaled=Voxel3D.shape
    
    XYZ_SampleStruct = {
        'X_sample': np.arange(Dimension_BBox_scaled[0] + 1),
        'Y_sample': np.arange(Dimension_BBox_scaled[1] + 1),
        'Z_sample': np.arange(Dimension_BBox_scaled[2] + 1)
    }
    
    [VolumeMass,Center,_]=z.calculate_bbox_moment(Voxel3D,1,XYZ_SampleStruct)
    
    [AverageVoxelDist2Center,_]=z.calculate_molecular_radius(Voxel3D,Center,VolumeMass,Param['default_radius_multiplier'])
        
    SphereXYZ_SampleStruct=z.get_bbox_moment_xyz_sample(Center,AverageVoxelDist2Center,Dimension_BBox_scaled)
    
    _,_,SphereBBoxMoment=z.calculate_bbox_moment(Voxel3D,MaxOrder,SphereXYZ_SampleStruct)
    
    [ZMoment_scaled, _]=z.calculate_bbox_moment_2_zm(MaxOrder
                                                  ,GCache_complex
                                                  ,GCache_pqr_linear
                                                  ,GCache_complex_index
                                                  ,CLMCache3D
                                                  ,SphereBBoxMoment)
    
    ZM_3DZD_invariant=z.get_3dzd_121_descriptor(ZMoment_scaled)
    
    ZM_3DZD_invariant_121=ZM_3DZD_invariant[~np.isnan(ZM_3DZD_invariant)]
    return ZM_3DZD_invariant_121


print(f"Merge all steps into a single callable NumPy function, OneTimeConversion.")


In [ ]:
%%time

Voxel3D=matrix = np.random.rand(100, 100, 100)
print(f"Initialize a 100x100x100 matrix with random values in CPU memory.")
Repeat=10000

for _ in range(Repeat):
    OneTimeConversion(Voxel3D,MaxOrder)

print(f"Iteratively invoke the OneTimeConversion function {Repeat} times using a for loop.")
print(f"Noted: Most NumPy runtimes are configured to use multithreading by default.")
print(f"Time elapsed is as follows:")